In [4]:
import pandas as pd
import numpy as np

ny_path = "/home/fuest/EnData/data/pecanstreet/15minute_data_newyork.csv"
austin_path = "/home/fuest/EnData/data/pecanstreet/15minute_data_austin.csv"
cali_path = "/home/fuest/EnData/data/pecanstreet/15minute_data_california.csv"
metadata_path = "/home/fuest/EnData/data/pecanstreet/metadata.csv"

ny_data = pd.read_csv(ny_path)
austin_data = pd.read_csv(austin_path)
cali_data = pd.read_csv(cali_path)
metadata = pd.read_csv(metadata_path)

ny_user_ids = ny_data.dataid.unique().tolist()
austin_user_ids = austin_data.dataid.unique().tolist()
cali_user_ids = cali_data.dataid.unique().tolist()

In [5]:
ny_data[ny_data.dataid == 27].to_csv("tests/testdata/15minute_data_newyork.csv")

OSError: Cannot save file into a non-existent directory: 'tests/testdata'

In [ ]:
all_user_ids = cali_user_ids + austin_user_ids + ny_user_ids

In [ ]:
metadata = metadata.loc[metadata["dataid"].isin(all_user_ids)].copy()
metadata["car1"].value_counts()

yes    15
Name: car1, dtype: int64

In [ ]:

USER_IDs = [27,  142,  203,  387,  558,  661,  914,  950, 1222, 1240, 1417,
       1450, 1524, 1642, 1731, 2096, 2318, 2335, 2358, 2361, 2606, 2818,
       3000, 3039, 3456, 3488, 3517, 3538, 3687, 3700, 3864, 3938, 3996,
       4031, 4283, 4373, 4495, 4550, 4767, 4934, 5058, 5587, 5679, 5746,
       5938, 5982, 5997, 6139, 6377, 6547, 7062, 7114, 7536, 7719, 7800,
       7901, 7951, 8061, 8156, 8342, 8386, 8565, 8574, 8733, 9019, 9053,
       9160, 9213, 9278, 9612, 9775, 9836, 9922]


In [3]:
from datasets.pecanstreet import PecanStreetDataManager

pv = PecanStreetDataManager(geography="austin", include_generation=True).create_all_pv_user_dataset()
non_pv = PecanStreetDataManager(geography="austin",include_generation=True).create_non_pv_user_dataset()

In [14]:
from datasets.timeseries_dataset import TimeSeriesDataset
from generator.data_generator import DataGenerator

df = pv.data
df

generator = DataGenerator("acgan", model_params=None)

conditioning_dict = {
    "month": 12,
    "weekday": 7,
    "city": 6,
    "building_type": 3,
    "has_solar": 2,
    "car1": 2,
    "state": 3,
    "total_square_footage": 5,
    "house_construction_year": 5
}

dataset = TimeSeriesDataset(df, "timeseries", conditioning_dict)
generator.fit(dataset, "timeseries")

Epoch 1:  62%|██████▏   | 133/213 [00:06<00:03, 20.05it/s]


KeyboardInterrupt: 